<a href="https://colab.research.google.com/github/Onamihoang/NLP-IELTS/blob/master/Try_FitBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FitBert

[FitBert](https://github.com/Qordobacode/fitbert) ((F)ill (i)n (t)he blanks, (BERT)) is a library for using BERT to fill in the blank(s) in a section of text from a list of options.

It's easy to use, just install with pip:

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!pip install fitbert

     |████████████████████████████████| 225kB 2.8MB/s 
     |████████████████████████████████| 573kB 8.8MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 890kB 14.1MB/s 
     |████████████████████████████████| 3.7MB 17.3MB/s 
     |████████████████████████████████| 1.0MB 37.2MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
  Created wheel for fitbert: filename=fitbert-0.7.0-cp36-none-any.whl size=214149 sha256=b4904100e2f8d194261d713d05e3e5a4ea586db7ed2297752a2ac937ec05a777
  Stored in directory: /root/.cache/pip/wheels/49/5e/fa/3c4789d2a679de4119357f14d28843a1262c3d5d1cf96807d7
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e21a3247ae076f713d2f7cac54b9b150741ed2365a3e3045805c79ea64f4d36b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for dill: filename=dill-0.2.5-cp36-none-any.whl size=52421 sha256=37d

Then import and use it (note - this requires downloading and loading into memory a pretrained BERT model and takes a minute or two):

In [3]:
from fitbert import FitBert


# in theory you can pass a model_name and tokenizer, but currently only
# bert-large-uncased and BertTokenizer are available
# this takes a while and loads a whole big BERT into memory
fb = FitBert()

masked_string = "Why Bert, you're looking ***mask*** today!"
options = ['buff', 'handsome', 'strong']

ranked_options = fb.rank(masked_string, options=options)
ranked_options

using model: bert-large-uncased
device: cuda


['handsome', 'strong', 'buff']

In [4]:
filled_in = fb.fitb(masked_string, options=options)
filled_in

"Why Bert, you're looking handsome today!"

There's a convenience method for masking a span (and filling in the suggestion, or not):

In [5]:
unmasked_string = "Why Bert, you're looks handsome today!"
span_to_mask = (17, 22)

filled_in = fb.mask_fitb(unmasked_string, span_to_mask)
filled_in

"Why Bert, you're  looking  handsome today!"

In [6]:
masked_string, masked = fb.mask(unmasked_string, span_to_mask)
print(masked_string, masked)

Why Bert, you're  ***mask***  handsome today! looks


## From the "Introducing FitBERT" blog post

### SWE section

In [7]:
masked_string = "Your 6 ***mask*** sodas are on their way !"
options = ['hot', 'cold', 'sweet', 'delicious', 'artisanal']
fb.fitb(masked_string, options=options)

'Your 6 cold sodas are on their way !'

In [8]:
masked_string = "Your 17 ***mask*** burritos are on their way !"
options = ['hot', 'cold', 'sweet', 'delicious', 'artisanal']
fb.fitb(masked_string, options=options)

'Your 17 delicious burritos are on their way !'

### Researcher section

One use case for FitBERT is easily evaluating the syntactic capabilities of any model available through the [Transformers library](https://medium.com/r/?url=https%3A%2F%2Fgithub.com%2Fhuggingface%2Ftransformers), which includes BERT, RoBERTa, GPT2, and DistillBERT.

This is very similar to Yoav GoldBerg's [Assessing BERT's Syntactic Abilities](https://arxiv.org/abs/1901.05287). AFAIK, this experiment hasn't been repeated with RoBERTa or DistillBERT, but would be interesting.

In [9]:
# example from "Targeted Syntactic Evaluation of Language Models"
# https://arxiv.org/abs/1808.09031

masked_string = "the author that the guard likes ***mask***"
options = ['laugh', 'laughs']
fb.rank_with_prob(masked_string, options)

(['laughs', 'laugh'], [4.141925951522962e-12, 3.37479962595269e-13])

In [10]:
# example from "Assessing the Ability of LSTMs to Learn Syntax-Sensitive Dependencies"
# https://transacl.org/ojs/index.php/tacl/article/view/972

masked_string = "accusations of abusive sockpuppetry from a trusted source ***mask*** a serious chilling effect ."
options = ["have", "has"]
fb.rank_with_prob(masked_string, options)

(['have', 'has'], [0.889950156211853, 0.004103285726159811])

### Using FitBERT with a spell corrector

Example of refining the output of a [word-vector-based spell checker](https://blog.usejournal.com/a-simple-spell-checker-built-from-word-vectors-9f28452b6f26) with BERT. This would also work with something like Hunspell.

In [11]:
input = "We predict the following issues will ocur."
masked_string = "We predict the following issues will ***mask*** ."
# mispelling vector subtraction gives the following options for "ocur"
options = ['ocur', 'occur', 'arise', 'happen', 'reliably']
fb.fitb(masked_string, options=options)

'We predict the following issues will arise .'

In [12]:
input = "We predict the following issues will ocur."
masked_string = "We predict the following issues will ***mask*** ."
# mispelling vector subtraction gives the following options for "ocur", but filter through Levenshtein distance threshold:
options = ['ocur', 'occur']
fb.fitb(masked_string, options=options)

'We predict the following issues will occur .'

### (Work in Progress) Using FiTBERT for truecasing

An efficient implementation will require:

1. Fixing the bug where probabilities returned by `fb.rank(with_prob=True)` aren't in the same order as the tokens returned
2. Tensorizing the handling of multi-token masks

In [0]:
from transformers import *

In [14]:
new_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
new_bert = BertForMaskedLM.from_pretrained('bert-base-cased')

fb2 = FitBert(model=new_bert, tokenizer=new_tokenizer, disable_gpu=True)


using model: bert-large-uncased
device: cpu


In [0]:
def change_case(word: str):
    if not word.isalpha():
        return False

    if word.lower() == word:
        return word.capitalize()
    elif word.capitalize() == word:
        return word.lower()
    else:
        # camelCase, ALLUPPER, sPoNGeBoB, etc
        return False

In [16]:
# Naive implementation handles some cases
masked_string = f'{fb2.mask_token} more than 2000 minerals are known, nearly all rocks are formed from seven mineral groups.'
'''
#(A) Although 
(B) However 
(C) Despite 
(D) Since   
'''
#masked_string = f"These {fb2.mask_token} some common grammatical mistakes ."
options = ["Although", "However", "Despite", "Since" ]
fb.rank_with_prob(masked_string, options=options)


(['Although', 'Since', 'Despite', 'However'],
 [0.7068151831626892,
  0.007502363063395023,
  0.0011061099357903004,
  0.00021216865570750087])

In [17]:
masked_string = f"Our friends are expected to assume the burden of their own defense,{fb2.mask_token} they are competent to do."
options = ["which we are certain", "that we are certain of ", "of which we are sure", "for which we are sure" ]
fb.rank_with_prob(masked_string, options=options)

(['which we are certain',
  'of which we are sure',
  'for which we are sure',
  'that we are certain of'],
 [7.986056449143198e-14,
  8.050692507159841e-15,
  3.7081823354854364e-15,
  5.0975905690246776e-21])

In [18]:
# but not others
masked_string = f"These are some Common {fb2.mask_token} mistakes ."
options = ["grammatical", "Grammatical"]
fb2.fitb(masked_string, options=options)


'These are some Common Grammatical mistakes .'

In [19]:
# we can loop through and look at each pairwise comparison to see whats going on
# except we can't trust the probabilities order, because of a bug in fitbert
orig = "These are some Common Grammatical mistakes ."

tokens = orig.split(" ")
for i, token in enumerate(tokens):
    masked_string = " ".join(tokens[:i]) + fb2.mask_token + " ".join(tokens[i:])
    changed = change_case(token)
    if changed:
        options = [changed, token]
        ranked, probs = fb2.rank(masked_string, options, with_prob=True)
        print(ranked, probs)

['These', 'these'] [0.00013668806059285998, 1.906781290017534e-05]
['are', 'Are'] [0.0012963489862158895, 1.4108933328316198e-06]
['some', 'Some'] [0.0023925018031150103, 9.76750652625924e-06]
['common', 'Common'] [0.05046171322464943, 0.02603953145444393]
['Grammatical', 'grammatical'] [0.000162732059834525, 3.9787528294255026e-06]
['mistakes', 'Mistakes'] [0.00013252747885417193, 2.397526372988068e-07]


In [20]:
# Using greedy decoding works pretty well

orig = "These are some Common Grammatical mistakes ."

tokens = orig.split(" ")
for i, token in enumerate(tokens):
    if token.isalpha():
        masked_string = " ".join(tokens[0:i]) + " " + fb2.mask_token + " " + " ".join(tokens[i + 1:])
        print("the masked string is: ", masked_string)
        changed = change_case(token)
        if changed:
            options = [changed, token]
            ranked, probs = fb2.rank(masked_string, options, with_prob=True)
            print(ranked, probs)
            if ranked[0] == changed:
                # should use probs, but there is a bug where it is sorted before being returned :facepalm:
                tokens[i] = changed
                print("the string is now:", " ".join(tokens))
print("final version is")
print(" ".join(tokens))

the masked string is:   ***mask*** are some Common Grammatical mistakes .
['These', 'these'] [0.05224079638719559, 0.0011475178180262446]
the masked string is:  These ***mask*** some Common Grammatical mistakes .
['are', 'Are'] [0.4918549060821533, 0.00012340431567281485]
the masked string is:  These are ***mask*** Common Grammatical mistakes .
['some', 'Some'] [0.006553384475409985, 0.0004972806782461703]
the masked string is:  These are some ***mask*** Grammatical mistakes .
['common', 'Common'] [0.22821259498596191, 0.011387941427528858]
the string is now: These are some common Grammatical mistakes .
the masked string is:  These are some common ***mask*** mistakes .
['grammatical', 'Grammatical'] [1.658693508943543e-05, 1.957110725925304e-06]
the string is now: These are some common grammatical mistakes .
the masked string is:  These are some common grammatical ***mask*** .
['mistakes', 'Mistakes'] [0.0025533486623317003, 5.693044258237023e-09]
final version is
These are some common

In [21]:
# I can't tell if this behaviour is ok
# Wikipedia says that this correction is good
# I thought this would be hard for the model...

orig = "I 'm really feeling Panic! At The Disco ."

tokens = orig.split(" ")
for i, token in enumerate(tokens):
    if i>0:
        if token.isalpha():
            masked_string = " ".join(tokens[0:i]) + " " + fb2.mask_token + " " + " ".join(tokens[i + 1:])
            changed = change_case(token)
            if changed:
                options = [changed, token]
                ranked, probs = fb2.rank(masked_string, options, with_prob=True)
                if ranked[0] == changed:
                    # should use probs, but there is a bug where it is sorted before being returned :facepalm:
                    tokens[i] = changed
print("final version is")
print(" ".join(tokens))

final version is
I 'm really feeling Panic! at the Disco .


In [22]:
# Truecasing is nearly impossible if a product name is also a common noun

orig = "Create Styleguides to standardize a writing style across all your content — or to manage distinct styles for different audiences ."

tokens = orig.split(" ")
for i, token in enumerate(tokens):
    if token.isalpha() and i>0:
        masked_string = " ".join(tokens[0:i]) + " " + fb2.mask_token + " " + " ".join(tokens[i + 1:])
        changed = change_case(token)
        if changed:
            options = [changed, token]
            ranked, probs = fb2.rank(masked_string, options, with_prob=True)
            if ranked[0] == changed:
                # should use probs, but there is a bug where it is sorted before being returned :facepalm:
                tokens[i] = changed
print("final version is")
print(" ".join(tokens))

final version is
Create styleguides to standardize a writing style across all your content — or to manage distinct styles for different audiences .
